In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:

!ls drive/My\ Drive

 att10.gdoc   att12.gdoc   att8.gdoc	  'Colab Notebooks'	   NAEPdata
 att11.gdoc   att6.gdoc    attfinal.gdoc  'data for nlp project'   soln10.js


In [3]:
!pip install transformers

import csv

def read_davison_data():
    reader = csv.reader(open("/content/drive/My Drive/data for nlp project/labeled_data.csv"))
    tweets = []
    labels = []
    firstline = True
    for i in reader:
        if firstline:
            firstline = False
            continue
        tweets.append(i[-1])
        labels.append(int(not (int(i[-2]) == 2)))
    return tweets, labels

def read_olid_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/olid-training-v1.0.tsv"),delimiter="\t")
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(int(i[2]=="OFF"))
  return tweets, labels



dtweets, dlabels = read_davison_data()
otweets, olabels = read_olid_data()

import torch
print(torch.cuda.is_available())
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments


train_texts, test_texts, train_labels, test_labels = train_test_split(dtweets+otweets, dlabels+olabels, test_size=.2, random_state=42)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)


class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = HateSpeechDataset(train_encodings, train_labels)
test_dataset = HateSpeechDataset(test_encodings, test_labels)

print("data loaded")

     |████████████████████████████████| 1.4MB 13.6MB/s 
     |████████████████████████████████| 890kB 47.8MB/s 
     |████████████████████████████████| 2.9MB 43.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e8b3c8168979034e13c224d93ffc50f21cd53879d7bdd1e37d5b08951ac515d7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
True



data loaded


In [4]:
#!pip install transformers


training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data for nlp project/results",          # output directory
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="/content/drive/My Drive/data for nlp project/logs",              # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
#model = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeech/")
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,             # evaluation dataset
    compute_metrics=compute_metrics
)




Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
trainer.train()

model.save_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeech/")
print("model saved")

Step,Training Loss
10,0.662308
20,0.644548
30,0.657378
40,0.645064
50,0.626730
60,0.598777
70,0.596981
80,0.554413
90,0.505635
100,0.485573


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeech/",num_labels=2)
device = torch.device('cuda')
model.to(device)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

#the following functions taken from brian zylich
def compute_f1(y_pred, y_true, epsilon=1e-7):
    tp = (y_true * y_pred).sum().to(torch.float32)
    # tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)

    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    f1 = 2 * (precision * recall) / (precision + recall + epsilon)
    return float(f1)

def evaluate(model, batch_loader):
    model.eval()
    with torch.no_grad():
        all_labels = None
        all_logits = None
        for batch in batch_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, return_dict=True)
            logits = outputs.logits

            if all_labels is None:
                all_labels = labels
                all_logits = logits
            else:
                all_labels = torch.cat((all_labels, labels), dim=0)
                all_logits = torch.cat((all_logits, logits), dim=0)

        loss_fct = torch.nn.CrossEntropyLoss()
        test_loss = loss_fct(all_logits.view(-1, 2), all_labels.view(-1))
        test_loss = float(test_loss)

        all_probs = torch.nn.functional.softmax(all_logits, dim=1)
        all_labels_hat = all_probs.argmax(dim=1)

        test_f1 = compute_f1(all_labels_hat, all_labels)

    return test_loss, test_f1

test_loss, test_f1 = evaluate(model, torch.utils.data.DataLoader(test_dataset))
print("test loss:", test_loss, "test f1:", test_f1, flush=True)


In [ ]:
#load model
from transformers import DistilBertModel

#model = DistilBertModel.from_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeech/")
#model.save_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeechnoclassification/")
model = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeechnoclassification/",num_labels=1)

def read_competition_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/train.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_texts, train_labels = read_competition_data()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

train_dataset = HateSpeechDataset(train_encodings, train_labels)


training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data for nlp project/results",          # output directory
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="/content/drive/My Drive/data for nlp project/logs",              # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

trainer.train()

model.save_pretrained("/content/drive/My Drive/data for nlp project/trainedoncompdata/")
print("model saved")

In [ ]:
import torch

def read_competition_dev_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/public_dev.csv"))
  tweets = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
  return tweets
  
encodings = tokenizer(read_competition_dev_data(), truncation=True, padding=True)
print(encodings)

device = torch.device('cuda')

intokens = torch.tensor(encodings["input_ids"]).to(device)

print(intokens.size())

with torch.no_grad():
  model.eval()

  out = model(intokens)
  print(out[0].size())
  with open('/content/drive/My Drive/data for nlp project/output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["id","offense_rating"])
    for i in range(1000):
      towrite = max(0,float(out[0][i][0]))
      writer.writerow([str(i+8001),towrite-(towrite%.001)])
print("output written")

In [ ]:
#trains base distilbert on our training set and evaluates on our test set
seed=3

import torch
print(torch.cuda.is_available())
from sklearn.metrics import mean_squared_error
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

def read_train_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/train_90.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

def read_test_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/test_10.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data for nlp project/results",  
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    eval_steps = 100,
    evaluation_strategy = "steps",
    seed=seed,
)

train_texts, train_labels = read_train_data()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

train_dataset = HateSpeechDataset(train_encodings, train_labels)

eval_texts, eval_labels = read_test_data()

eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)

eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    return {"rootMSE":mean_squared_error(preds,labels)**.5,}

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()
model.save_pretrained("/content/drive/My Drive/data for nlp project/distilbertseed"+str(seed)+"/")
print("model saved")

True


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Step,Training Loss,Validation Loss,Rootmse
100,1.080448,0.786749,0.886989
200,0.525876,0.523693,0.723667
300,0.588281,0.439072,0.662625
400,0.519266,0.421891,0.649531
500,0.271417,0.553311,0.743849
600,0.369702,0.400575,0.632910
700,0.311472,0.369136,0.607566
800,0.235162,0.426096,0.652760
900,0.262769,0.359245,0.599370
1000,0.195428,0.380800,0.617090


In [ ]:
#trains pretrained (on olid/davison) model on our training set and evaluates on our test set
import torch
print(torch.cuda.is_available())
from sklearn.metrics import mean_squared_error
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilberttrainedonhatespeechnoclassification/",num_labels=1)


def read_train_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/train_90.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

def read_test_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/test_10.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data for nlp project/results",  
    num_train_epochs=30,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    eval_steps = 100,
    evaluation_strategy = "steps"
)

train_texts, train_labels = read_train_data()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

train_dataset = HateSpeechDataset(train_encodings, train_labels)

eval_texts, eval_labels = read_test_data()

eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)

eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    return {"rootMSE":mean_squared_error(preds,labels)**.5,}

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()

True


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/My Drive/data for nlp project/distilberttrainedonhatespeechnoclassification/ and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Rootmse
100,0.844961,0.757809,0.870522
200,0.637354,0.540356,0.735089
300,0.499034,0.449257,0.670266
400,0.380975,0.495814,0.704141
500,0.348343,0.436557,0.660725
600,0.232169,0.450727,0.671362
700,0.350992,0.418979,0.647286
800,0.302554,0.430496,0.656122
900,0.353250,0.410261,0.640516
1000,0.148929,0.366036,0.605009


TrainOutput(global_step=13500, training_loss=0.056484492549189816)

In [ ]:
#trains base bert on our training set and evaluates on our test set
!pip install transformers
import torch
import csv
print(torch.cuda.is_available())
from sklearn.metrics import mean_squared_error
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

def read_train_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/train_90.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

def read_test_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/test_10.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data for nlp project/results",  
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    eval_steps = 100,
    evaluation_strategy = "steps"
)

train_texts, train_labels = read_train_data()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

train_dataset = HateSpeechDataset(train_encodings, train_labels)

eval_texts, eval_labels = read_test_data()

eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)

eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    return {"rootMSE":mean_squared_error(preds,labels)**.5,}

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()

True


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Step,Training Loss,Validation Loss,Rootmse
100,0.869964,0.720861,0.849035
200,0.588235,0.548010,0.740277
300,0.457591,0.451635,0.672038
400,0.430214,0.405435,0.636738
500,0.315631,0.466945,0.683334
600,0.284384,0.367663,0.606353
700,0.427899,0.421793,0.649456
800,0.247906,0.377993,0.614811
900,0.284509,0.345514,0.587804
1000,0.274866,0.357170,0.597637


TrainOutput(global_step=2700, training_loss=0.2149694598162616)

In [ ]:
#trains rberta on our training set and evaluates on our test set
!pip install transformers
import torch
import csv
print(torch.cuda.is_available())
from sklearn.metrics import mean_squared_error
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification, Trainer, TrainingArguments

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=1)

def read_train_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/train_90.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

def read_test_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/test_10.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

training_args = TrainingArguments(
    output_dir="/content/drive/My Drive/data for nlp project/results",  
    num_train_epochs=6,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    eval_steps = 100,
    evaluation_strategy = "steps"
)

train_texts, train_labels = read_train_data()

train_encodings = tokenizer(train_texts, truncation=True, padding=True)

train_dataset = HateSpeechDataset(train_encodings, train_labels)

eval_texts, eval_labels = read_test_data()

eval_encodings = tokenizer(eval_texts, truncation=True, padding=True)

eval_dataset = HateSpeechDataset(eval_encodings, eval_labels)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    return {"rootMSE":mean_squared_error(preds,labels)**.5,}

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,

)

trainer.train()

True


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Step,Training Loss,Validation Loss,Rootmse
100,0.932391,0.863525,0.929261
200,0.476941,0.501824,0.708396
300,0.407304,0.445511,0.667466
400,0.637598,0.691965,0.831844
500,0.322745,0.506797,0.711896
600,0.332727,0.393456,0.627261
700,0.560321,0.452256,0.672500
800,0.432812,0.445683,0.667595
900,0.329529,0.411532,0.641507
1000,0.265607,0.362247,0.601870


TrainOutput(global_step=2700, training_loss=0.2693367286964699)

In [ ]:
#loads several distilberts and ensembles
import torch
!pip install transformers
import csv
print(torch.cuda.is_available())
from sklearn.metrics import mean_squared_error
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments


tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model1 = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilbertseed0/")
model2 = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilbertseed1/")
model3 = DistilBertForSequenceClassification.from_pretrained("/content/drive/My Drive/data for nlp project/distilbertseed2/")
model4 = model


def read_test_data():
  reader = csv.reader(open("/content/drive/My Drive/data for nlp project/test_10.csv"))
  tweets = []
  labels = []
  firstline = True
  for i in reader:
    if firstline:
      firstline = False
      continue
    tweets.append(i[1])
    labels.append(float(i[5]))
  return tweets, labels

tweets,labels = read_test_data()
encodings = tokenizer(tweets, truncation=True, padding=True)
print(encodings)

device = torch.device('cuda')

model1.to(device)
model2.to(device)
model3.to(device)
model4.to(device)

intokens = torch.tensor(encodings["input_ids"]).to(device)

print(intokens.size())

labels = torch.tensor(labels)


with torch.no_grad():
  model1.eval()
  model2.eval()
  model3.eval()
  model4.eval()

  (out,) = model4(intokens)
  out = out.resize(800)
  print(mean_squared_error(labels,out.cpu())**.5)
  print(labels.size())
  
  


True
{'input_ids': [[101, 1000, 2043, 2009, 1005, 1055, 2051, 2000, 2552, 1998, 2017, 1005, 2128, 2035, 2894, 2007, 2115, 2067, 2114, 1996, 2813, 1010, 1996, 2069, 2376, 2008, 5609, 2003, 1996, 2028, 1999, 2115, 2132, 1012, 1000, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1045, 3062, 1999, 2293, 2007, 1037, 2611, 2040, 2069, 2354, 2176, 15260, 2016, 2134, 1005, 1056, 2113, 1045, 5839, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:447: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


0.9986731249713334
torch.Size([800])
